In [24]:
# Initializing

# data processing. pandas as alias pd
import pandas as pd 
# linear algebra. numpy as alias np
import numpy as np 

# If you're working with a notebook, don't forget to use Matplotlib magic! 
%matplotlib inline
# matlab-style plotting
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
# Set the Seaborn theme if desired
sns.set_style("darkgrid")

# ignore sklearn & seaborn warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn 

#for some statistics
from scipy import stats
from scipy.stats import norm, skew  #ex. sns(fit = norm)


# Format scientific notation from pandas aggregation
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000) 

In [25]:
df_dummies = pd.read_csv('data/all_data.csv')
df_no_dummies = pd.read_csv('data/all_data_no_dummies.csv') 

In [26]:
targets = pd.read_csv('data/y_train.csv') 

In [48]:
from sklearn.model_selection import GridSearchCV, train_test_split 

from sklearn import linear_model as lm 
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr

from sklearn.metrics import mean_squared_error  

#rmse error: 
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### Split all_data into train and test: 

In [52]:
ntrain = targets.shape[0] 
x_d = df_dummies[:ntrain] 
x_nd = df_no_dummies[:ntrain] 
targets_log = targets.apply(np.log)


In [44]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
scaler.fit(x_d)
x_d_std = scaler.transform(x_d)


### Split x_ and x_nd + targets into train and test using train_test_split: 

In [58]:
x_d_train, x_d_test, y_d_train, y_d_test = train_test_split(x_d_std, targets_log, test_size = 0.2, random_state = 42) 
x_nd_train, x_nd_test, y_nd_train, y_nd_test = train_test_split(x_nd, targets_log, test_size = 0.2, random_state = 42)  


## MODELLING

In [102]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

### 1. Multiple Linear Regression: 

In [59]:
lm_model =lm.LinearRegression()
lm_model.fit(x_d_train, y_d_train)

#RMS error 
print("RMSE train: {}".format(rmse(y_d_train, lm_model.predict(x_d_train)))) 
print("RMSE test : {}".format(rmse(y_d_test,  lm_model.predict(x_d_test)))) 
print('R^2 score: {}'.format(lm_model.score(x_d_train, y_d_train)))




RMSE train: 0.17318243900299812
RMSE test : 15562705841228.213
R^2 score: 0.8014776983376941


### 2. Ridge/Lasso/Elastic Net : Penalised LR 

In [70]:
#1. ridge: 

grid_param = [{'alpha': np.linspace(1e-3,20,20)}]

gs = GridSearchCV(estimator=lm.Ridge(random_state=42), param_grid=grid_param, cv=5)

gs.fit(x_d_std, targets_log)

#cv_results_, grid_scores_ (to obsolete), best_estimator_, best_params_, best_score_
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_))
#print('')
model = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_d_train, model.predict(x_d_train))))
print("RMSE test : {}".format(rmse(y_d_test,  model.predict(x_d_test))))
print('R^2 score: {}'.format(model.score(x_d_train, y_d_train))) 

Best params: {'alpha': 20.0}
Best score : 0.7480641187701964
RMSE train: 0.17316473428517343
RMSE test : 0.18610748181507164
R^2 score: 0.8015182867679322


In [73]:
#2. Lasso: 

grid_param = [{'alpha': np.linspace(1e-3,20,20)}] 

gs = GridSearchCV(estimator=lm.Lasso(random_state= 42, normalize=False), param_grid=grid_param, cv=5)

gs.fit(x_d_std, targets_log)

#cv_results_, grid_scores_ (to obsolete), best_estimator_, best_params_, best_score_
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_))
#print('')
model = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_d_train, model.predict(x_d_train))))
print("RMSE test : {}".format(rmse(y_d_test,  model.predict(x_d_test)))) 


Best params: {'alpha': 0.001}
Best score : 0.7493747517752001
RMSE train: 0.17342910634294778
RMSE test : 0.18603750059370833


In [75]:
model.fit(x_d, targets_log)
print('The intercept is %.4f' %(model.intercept_))
lassoCoef = pd.Series(model.coef_, index=x_d.columns)
print('The slopes are \n%s' %(lassoCoef))  

The intercept is 5.6668
The slopes are 
ExterQual              -0.000
Fireplaces              0.041
GarageArea              0.000
GrLivArea               0.000
HeatingQC               0.014
KitchenQual             0.036
OverallQual             0.063
TotRmsAbvGrd            0.000
Num_Bathrooms           0.034
HouseSF                 0.000
YearRemodAgg            0.003
Exterior1st_AsbShng    -0.016
Exterior1st_AsphShn    -0.000
Exterior1st_BrkComm    -0.000
Exterior1st_BrkFace     0.027
Exterior1st_CBlock     -0.000
Exterior1st_CemntBd    -0.000
Exterior1st_HdBoard     0.008
Exterior1st_ImStucc     0.000
Exterior1st_MetalSd     0.003
Exterior1st_Plywood    -0.005
Exterior1st_Stone       0.000
Exterior1st_Stucco      0.007
Exterior1st_VinylSd    -0.011
Exterior1st_Wd Sdng     0.000
Exterior1st_WdShing    -0.000
Foundation_BrkTil      -0.000
Foundation_CBlock       0.000
Foundation_PConc        0.000
Foundation_Slab        -0.000
                        ...  
HouseStyle_SLvl         0.000


In [76]:
#3: Elastic Net 

grid_param = [{'alpha': np.linspace(1e-3,20,20), 'l1_ratio': np.linspace(0.015, 1, 20)}]

# Confirmed that setting scoring='neg_mean_squared_error' is the same result as using the default "R2" score.
gs = GridSearchCV(estimator=lm.ElasticNet(random_state=42), param_grid=grid_param, cv=5)

gs.fit(x_d_std, targets_log) 

#cv_results_, grid_scores_ (to obsolete), best_estimator_, best_params_, best_score_
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_)) 
#print('')
model = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_d_train, model.predict(x_d_train))))
print("RMSE test : {}".format(rmse(y_d_test, model.predict(x_d_test))))  

Best params: {'alpha': 0.001, 'l1_ratio': 1.0}
Best score : 0.7493747517752001
RMSE train: 0.17342910634294778
RMSE test : 0.18603750059370833


In [77]:
#proportion of coeffs > 1e-6 out of all coeffs 

sum(model.coef_ > 1e-6) / model.coef_.shape[0] 

0.4782608695652174

In [130]:

#test random forest regressor with default parameters: 

randomForest = ensemble.RandomForestRegressor()

# fit RF model on training set
randomForest.set_params(random_state=42)

randomForest.fit(x_d_train, y_d_train)  

print("The training error is: %.5f" % (1- randomForest.score(x_d_train, y_d_train))) 
print("The test     error is: %.5f" % (1 - randomForest.score(x_d_test, y_d_test)))   

The training error is: 0.04521
The test     error is: 0.33463


In [154]:
grid_para_forest = {
    'n_estimators': range(80, 200, 10), 
    'max_features': ['sqrt', 'auto']
} 

In [155]:
gs = GridSearchCV(estimator = RandomForestRegressor(\
                                    min_samples_split=5,\
                                    max_depth = 10,\
                                    min_samples_leaf=1,\
                                    random_state=42,\
                                    n_jobs=-1),\
                  param_grid = grid_para_forest, n_jobs=-1, cv=5)   

In [156]:
gs.fit(x_d_train, y_d_train)  

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': range(80, 200, 10), 'max_features': ['sqrt', 'auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [157]:
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_)) 

Best params: {'max_features': 'sqrt', 'n_estimators': 160}
Best score : 0.764677787370153


In [158]:
forest_final = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_d_train, forest_final.predict(x_d_train)))) #0.1305
print("RMSE test : {}".format(rmse(y_d_test,  forest_final.predict(x_d_test)))) #0.1996

RMSE train: 0.13034901054513245
RMSE test : 0.1996985958782946


In [166]:

forest_feature_importance = forest_final.feature_importances_

forest_sorted_features = sorted(list(zip(x_d.columns, forest_feature_importance)), key = lambda kv: kv[1], reverse = True) 

forest_sorted_features 

[('HouseSF', 0.1957805728715159),
 ('OverallQual', 0.11524353111401342),
 ('GrLivArea', 0.11318324858920066),
 ('YearRemodAgg', 0.0863273945490596),
 ('GarageArea', 0.0754268326421905),
 ('ExterQual', 0.0631267699654737),
 ('KitchenQual', 0.05701338811945293),
 ('Num_Bathrooms', 0.05291583533658713),
 ('Fireplaces', 0.04631053874420838),
 ('TotRmsAbvGrd', 0.029066905402628636),
 ('Foundation_PConc', 0.0272024047643721),
 ('HeatingQC', 0.01705453958977323),
 ('Foundation_CBlock', 0.009982056142071268),
 ('Neighborhood_NridgHt', 0.0084840666357218),
 ('Exterior1st_VinylSd', 0.007072095516051824),
 ('Neighborhood_OldTown', 0.0053175803548562925),
 ('Neighborhood_IDOTRR', 0.0051996861274895),
 ('HouseStyle_2Story', 0.0045782049445904464),
 ('HouseStyle_1Story', 0.00434797774142646),
 ('LotShape_Reg', 0.00427390561308247),
 ('Neighborhood_NAmes', 0.004145804289911111),
 ('Neighborhood_Crawfor', 0.0037701678207107056),
 ('Neighborhood_NoRidge', 0.0037021365648128245),
 ('Exterior1st_AsbShng'

In [ ]:
#graph feature importances 